In [ ]:
import pandas as pd
#load /content/train.csv
train=pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
test=pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')
train

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error


In [ ]:
#select features
train_features=train.iloc[:,1:101]
y=train['loss']
X_test=test.iloc[:,1:101]

In [ ]:
loss_count=train.groupby('loss').count()
#pie chart
plt.figure(figsize=(10,10))
plt.pie(loss_count['id'],labels=loss_count.index,autopct='%1.1f%%',shadow=True)
plt.title('Pie Chart of Loss')
plt.show()

In [ ]:
#normalize the value
scaler=StandardScaler()
train_features=scaler.fit_transform(train_features)
test_features=scaler.transform(X_test)


In [ ]:
#fit xgbregressor
xgb=XGBRegressor(tree_method='gpu_hist',  booster='gbtree',n_estimators=10000,learning_rate=0.01,max_depth=10)
#kfold
total_rmse=0
from sklearn.model_selection import KFold
kf=KFold(n_splits=10,random_state=42,shuffle=True)
for i,(train_index,test_index) in enumerate(kf.split(train_features,y)):
    x_train,y_train=train_features[train_index],y[train_index]
    x_test,y_test=train_features[test_index],y.iloc[test_index]
    xgb.fit(x_train,y_train,eval_set=[(x_test,y_test)],eval_metric='rmse',early_stopping_rounds=100,verbose=False)
    fold_rmse=np.sqrt(mean_squared_error(y_test,xgb.predict(x_test)))
    print("Fold#{} RMSE:{}".format(i,fold_rmse))
    total_rmse+=fold_rmse
print("Total RMSE:{}".format(total_rmse/5))

In [ ]:
p=pred=xgb.predict(train_features)


In [ ]:
mean_squared_error(p,y, squared=False)

In [ ]:
pred=xgb.predict(test_features)
submission=pd.DataFrame({'id':test['id'],'loss':pred})
submission.to_csv('submission3.csv',index=False)

Using tabnet
